## Final Project Submission

Please fill out:
* Student name: Evan Staffen
* Student pace: Full time
* Scheduled project review date/time: October 7th, 2022
* Instructor name: Joseph Mata
* Blog post URL: https://medium.com/@evan.staffen/why-i-chose-data-science-5bfedb7a695e


# Microsoft Movie Studios 
Evan Staffen

## Overview

This project analyzes data from both IMDB and Box Office Mojo, pulling data from over 2,000 movies over the years of 2010-2018. In an ever competitive market with many new streaming platforms, I used this data to make suggestions to help Microsoft successfully launce their brand new movie studio.

## Business Problem
Microsoft wants to open a new movie studio and venture into a field they have little to no experience in. While there are plenty of studios out there, Microsoft has the resources to cement themselves in the indusrty. In order to be successful from the start, I wanted to figure out which movies and genres have led to the highest earnings. As well, I wanted to see how the most successful studios are operating today. 

## Data Understanding
IMDB has almost 50 years of data showing movie ratings, their run times, the studios that made them and their genres. Box Office Mojo has data from the past 10 years showing movies and their gross both foreign and domestic. From these two datasets, I was able to make insights into specific movie genres and studios that have been successful and how they have done it.

## Import Necessary Modules

In [1]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly.subplots import make_subplots
import plotly.express as px

## Cleaning Movie Gross Dataset

In [3]:
#Download and read file
df_bom = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
df_bom

FileNotFoundError: [Errno 2] No such file or directory: 'zippedData/bom.movie_gross.csv.gz'

In [ ]:
df_bom.info()

Format so scientific notation is gone, drop all NAs because without foreign gross we cannot calculate total gross, and change foreign gross to a float so I can calculate total gross

In [ ]:
#Gets rid of sci-not
pd.set_option('display.float_format', lambda x: '%.9f' % x)

#Dropping NAs
dropped_df_bom = df_bom.dropna()

#Making foreign gross a float
dropped_df_bom['foreign_gross'] = dropped_df_bom.loc[:,'foreign_gross'].str.replace(',', '').astype(float)

#Calculate total gross
dropped_df_bom['total_gross'] = ((dropped_df_bom.loc[:,'domestic_gross'])+(dropped_df_bom.loc[:,'foreign_gross']))

dropped_df_bom

## Cleaning IMDB Dataset
After downloading the IMDB SQL database, I saw that movie_basics and movie_ratings shared the movie_id column so I joined them together

In [ ]:
conn = sqlite3.connect('zippedData/im.db')
cur = conn.cursor()
q = ("""
SELECT *
FROM movie_basics
""")
pd.read_sql(q,conn)

In [ ]:
conn = sqlite3.connect('zippedData/im.db')
cur = conn.cursor()
q = ("""
SELECT *
FROM movie_ratings
""")
pd.read_sql(q,conn)

Join the tables!

In [ ]:
conn = sqlite3.connect('zippedData/im.db')
cur = conn.cursor()
q = ("""
SELECT *
FROM movie_basics
JOIN movie_ratings
    USING(movie_id)
""")

sqldf = pd.read_sql(q,conn)
sqldf

This dataframe is missing many values in runtime, so I ultimately ended up dropping that.

In [ ]:
sqldf.info()

In [ ]:
sqldf.head(3)

After noticing that the genres column was difficult to interpret, I decided to organize it by separating each genre. After seeing a clear trend that action, animation, adventure, documentary and drama were the dominant genres, I decided to separate movies into just one category rather than IMDBs way of categorizing them. Afterwards, I set up the dataframe for merging by changing two column names to match with the Box Office Mojo dataset.

In [ ]:
#Editting genres column
sqldf_genres_drop = sqldf.dropna()
sqldf_genres_drop['genres'] = sqldf_genres_drop['genres'].map(lambda x:x.split(','))
sqldf_genres_drop['genres'].map(lambda x: str(x))

#Organizing new genres column
sqldf_genres_drop['genre1'] = sqldf_genres_drop['genres'].apply(lambda x: x[0])
sqldf_genres_drop['genre2'] = sqldf_genres_drop['genres'].apply(lambda x: x[1] if (len(x) == 2 or len(x) == 3) else '')
sqldf_genres_drop['genres_cleaned'] = sqldf_genres_drop['genre1'] + '-' + sqldf_genres_drop['genre2']
sqldf_genres_drop['genres_cleaned'] = sqldf_genres_drop['genres_cleaned'].apply(lambda x: x.rstrip('-') if x.endswith('-') else x)

#Action, Animation, Adventure, Documentary,Drama genres override other subgenres 
sqldf_genres_drop['genres_cleaned'] = sqldf_genres_drop['genres_cleaned'].apply(lambda x: x.replace(x,'Action') if x.endswith('-Action') or x.startswith('Action-') or x == ('Action') else x)
sqldf_genres_drop['genres_cleaned'] = sqldf_genres_drop['genres_cleaned'].apply(lambda x: x.replace(x,'Animation') if x.endswith('-Animation') or x.startswith('Animation-') or x == ('Animation') else x)
sqldf_genres_drop['genres_cleaned'] = sqldf_genres_drop['genres_cleaned'].apply(lambda x: x.replace(x,'Adventure') if x.endswith('-Adventure') or x.startswith('Adventure-') or x == ('Adventure') else x)
sqldf_genres_drop['genres_cleaned'] = sqldf_genres_drop['genres_cleaned'].apply(lambda x: x.replace(x,'Documentary') if x.endswith('-Documentary') or x.startswith('Documentary-') or x == ('Documentary') else x)
sqldf_genres_drop['genres_cleaned'] = sqldf_genres_drop['genres_cleaned'].apply(lambda x: x.replace(x,'Drama') if x.endswith('-Drama') or x.startswith('Drama-') or x == ('Drama') else x)

#renaming column for merge
sqldf_genres_drop.rename(columns = {'primary_title':'title','start_year':'year'},inplace=True)

sqldf_genres_drop

## Merging the Datasets for Analysis
The tables were merged on two columns, the title and year of the movie, only with records that they share.

In [ ]:
#Merge two tables on shared columns of title and year
merged_df = sqldf_genres_drop.merge(dropped_df_bom, how='inner',on=['title','year'])
merged_df


## Creating Layout for Analysis
From here, I knew that I wanted to see which genre of movies gross the most. I decided to organize my chart so that each row had the movie title, the genre, the studio that made it, as well as its total gross and rumtime.

In [ ]:
#Merging WB and WB NL into one studio, cleaning merged dataset for analysis
good_df = merged_df.groupby(['studio','genres_cleaned','title'],as_index = True)[['year','domestic_gross','foreign_gross','total_gross','runtime_minutes']].mean().sort_values(by=['total_gross'],ascending=[False])
no_index = good_df.reset_index()
no_index['studio'] = no_index['studio'].apply(lambda x:x.replace(x,'WB') if x == 'WB (NL)' else x)
no_index

In [ ]:
no_index.info()

In [ ]:
no_index.describe()

## Top 4 Grossing Genres Based on Top 100 Movies
After seeing that the 4 genres, action, animation, adventure and drama, are 96 of the top 100 movies, I plotted the total gross by genre and removed the other genres as they only had one movie in each.

In [ ]:
#Plot top 100 movies based on top 4 genres, as other genres only had one movie

movies = no_index.groupby(['title','genres_cleaned'])['total_gross'].sum().sort_values(ascending=False).reset_index().head(100)
movies_genre = movies.loc[(movies['genres_cleaned'] == 'Action')|(movies['genres_cleaned'] == 'Animation')|(movies['genres_cleaned'] == 'Adventure')|(movies['genres_cleaned'] == 'Drama')]
total_per_genre = movies_genre.groupby('genres_cleaned')['total_gross'].sum().sort_values(ascending=False).reset_index()
px.bar(total_per_genre,x='genres_cleaned',y='total_gross',title='Total Gross of Top 100 Movies Based on Genre',
      labels = {'genres_cleaned':'Genres','total_gross':'Total Gross'},
      color='genres_cleaned')




# Top 100 and Top 250 Grossing Movies by Genre


In [ ]:
#Organize movies by top 100 by total gross, use genres as colors
movies = no_index.groupby(['title','genres_cleaned'])['total_gross'].sum().sort_values(ascending=False).reset_index().head(100)
movies_genre = movies.loc[(movies['genres_cleaned'] == 'Action')|(movies['genres_cleaned'] == 'Animation')|(movies['genres_cleaned'] == 'Adventure')|(movies['genres_cleaned'] == 'Drama')]
totals = movies_genre.groupby('genres_cleaned').count().sort_values(by = 'total_gross',ascending=False).reset_index()
totals

fig, axes = plt.subplots(2,1, figsize = (6,10))

#Barplot
sns.barplot(ax=axes[0],data=totals,x='genres_cleaned',y='total_gross').set(xlabel ='Genres', ylabel= 'Movie Count', title = 'Top 100 Grossing Movies by Genre')

#Swarmplot
sns.boxplot(ax=axes[1],data=movies_genre,x='genres_cleaned',y='total_gross').set(xlabel ='Genres', ylabel= 'Total gross ($)', title = 'Top 100 Grossing Movies by Genre')

#list from value_counts by genre
plt.yticks([500000000,750000000,1000000000,1250000000,1500000000],['500M', '750M','1B', '1.25B','1.5B']);


In [ ]:
#Organize top 250 movies by title, genres and total gross
top250 = no_index.groupby(['title','genres_cleaned'])['total_gross'].sum().sort_values(ascending=False).reset_index().head(250)
top250_genres = top250.loc[(top250['genres_cleaned'] == 'Action')|(top250['genres_cleaned'] == 'Animation')|(top250['genres_cleaned'] == 'Adventure')|(top250['genres_cleaned'] == 'Drama')]
totals = top250_genres.groupby('genres_cleaned').count().sort_values(by='total_gross',ascending=False).reset_index()
totals
fig, axes = plt.subplots(2,1, figsize = (6,10))

#Barplot
sns.barplot(ax = axes[0],data=totals,x='genres_cleaned',y='total_gross').set(xlabel ='Genres', ylabel= 'Movie Count', title = 'Top 250 Grossing Movies by Genre')
#Swarmplot
sns.boxplot(ax = axes[1],data=top250_genres,x='genres_cleaned',y='total_gross').set(xlabel ='Genres', ylabel= 'Total gross ($)', title = 'Top 250 Grossing Movies by Genre')
#list of value_counts by genre
plt.yticks([250000000,500000000,750000000,1000000000,1250000000,1500000000],['250M','500M', '750M','1B', '1.25B','1.5B']);


## Top 10 Studios by Total Gross
I wanted to see who the top 10 studios were by total gross, to then delve deeper into if action or animation are leading to their success.

In [ ]:
#Top 10 Studios by gross
a = px.bar(no_index.groupby('studio')['total_gross'].sum().sort_values(ascending=False).reset_index().head(10),x='studio',y='total_gross',title = 'Top 10 Studios by Total Gross',labels = {'studio':'Studio','total_gross':'Total Gross ($)'},color = 'studio');
a

## Top 5 Studios Gross Proportions by Genre
I took the top 5 studios and separated the money they make from action, animation and other genre movies. From there I separated it into just action and other genres to demonstrate that in order to be a successful studio, you need 50% of your gross earnings coming from action movies.

In [ ]:
#Separate studios into top 5
new_no_index = no_index.groupby(['studio','genres_cleaned']).sum().sort_values(by='total_gross',ascending=False).reset_index()
for i,x in enumerate(new_no_index['studio']): 
    if x == 'BV': 
        new_no_index.loc[i,'studio'] = 'BV'
    elif x == 'Fox': 
        new_no_index.loc[i,'studio'] = 'Fox'
    elif x == 'Uni.':
        new_no_index.loc[i,'studio'] = 'Uni.'
    elif x == 'Par.':
        new_no_index.loc[i,'studio'] = 'Par.'
    elif x == 'WB':
        new_no_index.loc[i,'studio'] = 'WB'
    else:
        new_no_index.loc[i,'studio'] = 'other'

#get rid of all studios we don't want to analyze
new_no_index_deleted = new_no_index.loc[new_no_index['studio'] != 'other']

In [ ]:
new_no_index_deleted

#Bar plot showing all genres spread
m = px.bar(new_no_index_deleted,x='studio',y='total_gross',color='genres_cleaned',title='Total Gross by Studio Separated by Action, Animation and Other Genres',labels ={'total_gross':'Total Gross','studio':'Studio','genres_cleaned':'Genres'});
m

## Top 5 Studio Gross by Action Movies

In [ ]:
for i,x in enumerate(new_no_index_deleted['genres_cleaned']): 
    if x == 'Action': 
        new_no_index_deleted.loc[i,'genres_cleaned'] = 'Action'
    else: 
        new_no_index_deleted.loc[i,'genres_cleaned'] = 'Total'

new_no_index_deleted2 = new_no_index_deleted.loc[(new_no_index_deleted['genres_cleaned'] == 'Action')|(new_no_index_deleted['genres_cleaned'] == 'Total')]

r = px.bar(new_no_index_deleted2,x='studio',y='total_gross',color='genres_cleaned',title='Total Gross by Studio Separated by Action and Other Genres',labels ={'total_gross':'Total Gross','studio':'Studio','genres_cleaned':'Genres'})
r

## Runtime Action V Animation Movies
I wanted to see if movie runtimes had any influence on their gross earnings. There is a clear trend in that animation movies tend to be approximately 90 minutes and action movies are approximately 115 minutes.

In [ ]:
#compare runtime to total gross for action and animation movies
runtime = no_index.groupby(['genres_cleaned','year','studio'])[['runtime_minutes','total_gross']].mean().sort_values(by='total_gross',ascending=False)
runtime.reset_index(inplace=True)

#trying to do top 5 studios only
bv = runtime.loc[(runtime['studio'] == 'BV')&(runtime['genres_cleaned'] == 'Action') | (runtime['studio'] == 'BV') & (runtime['genres_cleaned']== 'Animation')]
wb = runtime.loc[(runtime['studio'] == 'WB')&(runtime['genres_cleaned'] == 'Action') | (runtime['studio'] == 'WB') & (runtime['genres_cleaned']== 'Animation')]
fox = runtime.loc[(runtime['studio'] == 'Fox')&(runtime['genres_cleaned'] == 'Action') | (runtime['studio'] == 'Fox') & (runtime['genres_cleaned']== 'Animation')]
uni = runtime.loc[(runtime['studio'] == 'Uni.')&(runtime['genres_cleaned'] == 'Action') | (runtime['studio'] == 'Uni.') & (runtime['genres_cleaned']== 'Animation')]
par = runtime.loc[(runtime['studio'] == 'Par.')&(runtime['genres_cleaned'] == 'Action') | (runtime['studio'] == 'Par.') & (runtime['genres_cleaned']== 'Animation')]
studio_runtime = pd.concat([bv,wb,fox,uni,par], axis = 0)

#studio_runtime


fig,axes = plt.subplots(1,1,figsize=(8,7))
#violin plots
sns.violinplot(data=studio_runtime, x="studio", y="runtime_minutes", hue="genres_cleaned",split=True).set(xlabel='Studios',ylabel='Runtime (min)',title='Average Runtime for Action and Animation movies by Studio')
plt.legend(title='Genres',bbox_to_anchor=(0.6,1));


I plotted the same graph with the median lines to demonstrate the fact that the top 5 studios tend to make their action movies longer than the industry as a whole.

In [ ]:
#With median times by top 5 studio and genre overall
fig, ax = plt.subplots(figsize=(8,7))
sns.violinplot(data=studio_runtime, x="studio", y="runtime_minutes", hue="genres_cleaned",split=True).set(xlabel='Studios',ylabel='Runtime (min)',title='Average Runtime for Action and Animation movies by Studio')
plt.legend(title='Genres')

#median lines
plt.axhline(y = 114, linestyle = '-',label='Median Top 5')
plt.axhline(y = 93, color = 'r',linestyle = '-',label = 'Median Top 5')
plt.axhline(y = 91, color = 'r', linestyle = '--',label = 'Median Overall')
plt.axhline(y = 104, color = 'b', linestyle = '--',label = 'Median Overall')
plt.legend(title='Genres',bbox_to_anchor =(1, 1));

## Conclusions

There are three key recommendations that I would make to Microsoft for their movie studios.
1. Ensure that action and animation movies are the primary focus of the studio.
2. In order to be a successful studio, at least 50% of the gross earnings need to come from action movies.
3. Animation and action movies tend to be around 90 and 110 minutes, although the top grossing action movies are significantly longer than the median.


## Next Steps

With further time and data I look further into:
1. Directors, writers and actors and how much their movies gross.
2. Investigate production budget by genre.
3. Consider how the movie landscape is changing with streaming services.